## Declaring API keys

In [ ]:
sarvamai_api_key = ""       # declare keys inside the quotes   
openai_api_key = ""

In [23]:
pip install --upgrade openai

   ---------------------------------------- 0.0/367.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/367.8 kB ? eta -:--:--
   --- ----------------------------------- 30.7/367.8 kB 660.6 kB/s eta 0:00:01
   ------- -------------------------------- 71.7/367.8 kB 1.3 MB/s eta 0:00:01
   --------------------- ------------------ 194.6/367.8 kB 1.7 MB/s eta 0:00:01
   -------------------------------- ------- 297.0/367.8 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 367.8/367.8 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/189.7 kB ? eta -:--:--
   ------------------------- -------------- 122.9/189.7 kB 3.6 MB/s eta 0:00:01
   ---------------------------------------- 189.7/189.7 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
!pip install requests
!pip install pyaudio


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\hp\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\hp\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [2]:
import pyaudio
import wave
import requests
import openai
import json
import base64

## Record Audio & STT

In [3]:

# Parameters for recording audio
FORMAT = pyaudio.paInt16  # Format of wave
CHANNELS = 1  # Number of channels (1 for mono)
RATE = 16000  # Sample rate (16kHz)
CHUNK = 1024  # Chunk size
RECORD_SECONDS = 10  # Duration to record (5 seconds)
WAVE_OUTPUT_FILENAME = "output.wav"  # Output file name


def record_stt():
    # Initialize PyAudio object
    audio = pyaudio.PyAudio()

    # Start recording
    print("Recording...")
    stream = audio.open(format=FORMAT, channels=CHANNELS,
                        rate=RATE, input=True,
                        frames_per_buffer=CHUNK)
    frames = []

    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    # Stop recording
    print("Finished recording.")
    stream.stop_stream()
    stream.close()
    audio.terminate()


    # Save the recorded audio as a WAV file
    wave_file = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wave_file.setnchannels(CHANNELS)
    wave_file.setsampwidth(audio.get_sample_size(FORMAT))
    wave_file.setframerate(RATE)
    wave_file.writeframes(b''.join(frames))
    wave_file.close()


    # Now, send the WAV file to the speech-to-text API (Hindi ASR)
    url = "https://api.sarvam.ai/speech-to-text-translate"
    
    api_key = sarvamai_api_key                      #Enter your API key

    # Open the recorded WAV file
    with open(WAVE_OUTPUT_FILENAME, 'rb') as audio_file:
        files = {
            'file': (WAVE_OUTPUT_FILENAME, audio_file, 'audio/wav'),
            "prompt": (None, "keep it generous and as it is and translate in english, if user speaks in HINDI."),
            "model": (None, "saaras:v1")
            #'language_code': (None, 'hi-IN')  # 'hi-IN' for Hindi language
        }

        headers = {
            "api-subscription-key": api_key,
        }

        # Send the POST request
        response = requests.post(url, headers=headers, files=files)

        # Get the transcribed text from the response
        transcribed_text = response.json()['transcript']  # Assuming the API returns JSON
        #print("Transcribed Text: ", transcribed_text)
        
        return transcribed_text







## Testing STT, jst run the cell and test the stt


In [13]:
output = record_stt()
print(output)

Recording...
Finished recording.
Hello hello hello hello hello hello hello.


## OpenAI API Call

In [4]:

from openai import OpenAI
openai_api_key = openai_api_key


prompt = "tell me about yourself."

def llm(prompt):
  client = OpenAI(
    api_key = openai_api_key,
  )

  stream = client.chat.completions.create(
            model = "gpt-4o-mini",
            messages = [{"role": "user", "content": prompt}],
            stream = True,
  )

  response = ""

  for chunk in stream:
      if chunk.choices[0].delta.content is not None:
          response += chunk.choices[0].delta.content
        
  return response
        
print(llm(prompt))

I am an AI language model created by OpenAI, designed to assist with a wide range of inquiries by providing information, answering questions, and engaging in conversation. My training includes a diverse dataset covering various topics, which allows me to generate responses that are informative and relevant. However, I don't have personal experiences, emotions, or opinions. My main goal is to assist you, so feel free to ask me anything!


## TTS

In [ ]:

tts_url = "https://api.sarvam.ai/text-to-speech"

message = ""               # write the sentence to be speak 

# Prepare payload for TTS API
tts_payload = {
    "inputs": [message],  # The LLM response must be wrapped in an array
    "target_language_code": "hi-IN",  # Hindi language code
    "speaker": "meera",  # Voice of the speaker, as per API docs
}

tts_headers = {
    "api-subscription-key": api_key,
    "Content-Type": "application/json"
}

# Send the POST request to TTS API
tts_response = requests.post(tts_url, json=tts_payload, headers=tts_headers)

# Process the response and save the audio file
if tts_response.status_code == 200:
    # Assuming the response contains base64-encoded audio data
    response_data = tts_response.json()
    audio_data_base64 = response_data['audios'][0]  # The first audio clip
    audio_file_name = "response_audio.wav"

    # Decode the base64 audio data and save it as a .wav file
    with open(audio_file_name, 'wb') as audio_file:
        audio_file.write(base64.b64decode(audio_data_base64))
    print(f"Audio saved to {audio_file_name}.")
else:
    print(f"Error: {tts_response.status_code}, {tts_response.text}")

# BOT

In [5]:
# defining intro prompt & other extraction fxns

intro_prompt = '''
### ROLE ###
You are given the role of an information collection bot for collecting users address related details.
This bot is designed to collect address and delivery preference information to ensure accurate and timely deliveries.
### INSTRUCTIONS ###
Be crisp in response.
'''




def extract_location_type(text):
  question = '''Hi, I am Sthaan Bot. Let's start by collecting your address and delivery preference information. Could you please tell me if you live in an apartment, a gated community, a village, or another type of location?'''
  prompt = f'''
  This is the question asked to user : {question}
  This is the response received from user : {text}

  ### STRICT INSTRUCTIONS ###
  You MUST give only one of the following response as plain text. I want no other word in your response.
  The options are :
  Apartment
  Gated Community
  Village
  Another type of location
  Not able to infer
  '''
  response = llm(intro_prompt + prompt)
  # print(response)
  return response




def isPhoneValid(s):
    # 1) Then contains 6,7 or 8 or 9.
    # 2) Then contains 9 digits
    if len(s)!=10:
      return False
    Pattern = re.compile("[6-9][0-9]{9}")
    return Pattern.match(s)



def validatePinCode(s):
    regexm = r"^[1-9][0-9]{5}$"
    return bool(re.match(regexm, s))


In [12]:
contact_json = {}
questions = [
    '''Hi, I am Sthaan Bot. Let's start by collecting your contact details. Can you tell me your name?''',
    '''Thank you. Can you please provide your contact number without country code?''',
]
json_format = [
    'Return JSON with key as "name". You must identify which part of the response is the name, YOU MUST NOT RETURN THINGS LIKE {"name":"my name is"} or {"name":"is"} if you cant retrive name then just return "Not Mentioned"',
    'Return JSON with key as "contact_number" and the datatype of key should be string not int.',
]
json_keys = [
    'name',
    'contact_number',
]

for i, question in enumerate(questions):
  count = 0
  while True:
    count+=1
    print('Bot: ' + ('Sorry I couldnt get that. ' if count>1 else '' ) + question, end='\n')
    
    # voice responce
    response = record_stt()
    
    #response = input("User: ")
    prompt = intro_prompt + f'''
    This is the question asked to the user: {question}
    This is the response received from user : {response}
    Instruction: {json_format[i]}
    ### STRICT INSTRUCTIONS ###
    You have to extract the information from the user's response according to the question, it is possible that user has not mentioned the required information in his response.
    You MUST give only the JSON as plain text. I DON'T WANY ANY OTHER WORD IN YOUR RESPONSE. Don't change the spelling of words used.
    WHEN THE USER PHONE NUMBER IS ASKED, STORE RESPONSE IN DIGITS, NOT IN WORDS.
    IF YOU CAN'T RETRIEVE THE INFORMATION OR THE USER HASN'T MENTIONED THE INFORMATION IN THE RESPONSE, GIVE THE KEY AS "Not Mentioned", REMEMBER THIS POINT THIS IS VERY IMPORTANT, WE DON'T WANT WRONG INFORMATION IN OUR DATABASE, MAKE SURE TO RESPOND 'Not Mentioned' IF YOU CANT RETRIEVE THE INFORMATION PROPERLY.
    '''
    response = llm(prompt)
    print(response)
    json_data = json.loads(response)
    contact_json[json_keys[i]] = 'Not Mentioned'
    if json_keys[i]=='contact_number':
      if not isPhoneValid(json_data[json_keys[i]]):
        continue
    if json_data[json_keys[i]] != 'Not Mentioned':
        contact_json[json_keys[i]] = json_data[json_keys[i]]
        break

contact_update_question = 'Bot: This is the information you provided, I will repeat them just for verification purposes, if you want to update anything then please let me know. The name you provided is ' + contact_json['name'] + ' and the contact number is ' + contact_json['contact_number'] + '. If you want to change anything then please let me know in which data do you want to make changes.'
print(contact_update_question, end='\n')

#voice response
contact_update_response = record_stt()

#contact_update_response = input("User: ")
contact_update_prompt = intro_prompt + f'''
This is the question asked to user: {contact_update_question}
The response from user: {contact_update_response}
This is the information in JSON format: {contact_json}
###INSTRUCTIONS###
You have to look at user's response and determine if they want to update any information.
If they dont want to update anything RETURN "NO" as your response, REMEMBER ONLY "NO", I WANT NO OTHER WORD IN YOUR RESPONSE.
If they want to update anything the a text in format of PYTHON LIST containing all the fields that need to be updated. For eg. if user wants to update his name and contact number then return ["name", "contact_number"], else if user wants to update his name only then return ["name"]. NO OTHER WORD OR SENTENCE SHOULD BE THERE IN YOUR RESPONSE OTHER THAN THIS PYTHON LIST.
'''
response = llm(contact_update_prompt)
print(response)
if response.strip() != 'NO':
  try:
    update_list = ast.literal_eval(response)
    for i in update_list:
      question = f'I heard you want to update your {i}. Please provide the updated {i}.'
      print(question, end='\n')
      
      # voice response
      
      response = record_stt()
      
      prompt = intro_prompt + f'''
      The user had previously given this information: {{'{i}':'{contact_json[i]}'}}
      But now he wants to update: {i}
      The question asked: {question}
      The response from user: {response}
      ### STRICT INSTRUCTIONS ###
      You have to extract the upadted information from response, and return it in form of JSON exactly it was before.
      You MUST give only the JSON as plain text. I DON'T WANY ANY OTHER WORD IN YOUR RESPONSE. Don't change the spelling of words used
      '''
      response = llm(prompt)
      # print(response)
      json_data = json.loads(response)
      contact_json[i] = json_data[i]

  except:
    pass


print('Bot: Thank you for providing the contact information.', end='\n')
print(contact_json, end='\n')

Bot: Hi, I am Sthaan Bot. Let's start by collecting your contact details. Can you tell me your name?
Recording...
Finished recording.
{"name":"Vivek Kumar Gupta"}
Bot: Thank you. Can you please provide your contact number without country code?
Recording...
Finished recording.
```json
{
    "contact_number": "9412424975"
}
```


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [13]:
info_json = {}
question = '''Bot: Could you please tell me if you live in an apartment, a gated community, a village, or another type of location?'''
print(question, end='\n')

#voice response
response = record_stt()

#response = input("User: ")
loc_type = extract_location_type(response)
info_json['location_type'] = loc_type

if loc_type.strip() == 'Apartment':
  questions = [
      'Thank you. Now we can move on to collect your address. Can you please your apartment number?',
      'Can you please provide the name of the tower or building?',
      'What is the name of the area or locality where it is situated?',
      'Can you tell me about any landmarks nearby, so that our delivery agents can find you more easily?',
      'What is the name of the city or town?',
      'What is the state?',
      'What is the pincode?',
      'Can you provide any delivery prefrences for example "Leave the package at door or with guard" if any?',
      'Can you provide the prefered time slot of your availability to collect the delivery?'
  ]
  json_format = [
      'Return JSON with key as "apartment_number"',
      'Return JSON with key as "building"',
      'Return JSON with key as "area"',
      'Return JSON with key as "landmarks" and value should be array containing all the landmarks',
      'Return JSON with key as "city"',
      'Return JSON with key as "state"',
      'Return JSON with key as "pincode"',
      'Return JSON with key as "delivery_preferences"',
      'Return JSON with key as "time_slot"'
  ]
  json_keys = [
      'apartment_number',
      'building',
      'area',
      'landmarks',
      'city',
      'state',
      'pincode',
      'delivery_preferences',
      'time_slot'
  ]

  for i, question in enumerate(questions):
    count = 0
    while True:
      count+=1
      print('Bot: ' + ('Sorry I couldnt get that. ' if count>1 else '' ) + question, end='\n')
      
      #voice response
      response = record_stt()
      
      #response = input("User: ", end='\n')
      prompt = intro_prompt + f'''
      This is the question asked to the user: {question + json_format[i]}
      This is the response received from user : {response}
      ### STRICT INSTRUCTIONS ###
      You have to extract the information from the user's response according to the question.
      You have to give the JSON containing the information given by the user.
      You MUST give only the JSON as plain text. I DON'T WANY ANY OTHER WORD IN YOUR RESPONSE. Don't change the spelling of words used.
      If you can't retrieve the information or the user hasn't mentioned the information in the response, give the key as "Not Mentioned", remember this point this is very important, we don't want wrong information in our database, make sure to respond not mentioned if you don't get the information properly.
      '''
      response = llm(prompt)
      print(response)
      
      json_data = json.loads(response)
      info_json[json_keys[i]] = 'Not Mentioned'
      if json_data[json_keys[i]] != 'Not Mentioned':
          info_json[json_keys[i]] = json_data[json_keys[i]]
          break

elif loc_type.strip() == 'Gated Community':
  questions = [
      'Thank you. Now we can move on to collect your address. Can you please your House number?',
      'Can you please provide the name of the gated community or the society?',
      'What is the name of the area or locality where it is situated?',
      'Can you tell me about any landmarks nearby, so that our delivery agents can find you more easily?',
      'What is the name of the city or town?',
      'What is the state?',
      'What is the pincode?',
      'Can you provide any delivery prefrences for example "Leave the package at door or with guard" if any',
      'Can you provide the prefered time slot of your availability to collect the delivery'
  ]
  json_format = [
      'Return JSON with key as "house_number"',
      'Return JSON with key as "gated_community"',
      'Return JSON with key as "area"',
      'Return JSON with key as "landmarks" and value should be array containing all the landmarks',
      'Return JSON with key as "city"',
      'Return JSON with key as "state"',
      'Return JSON with key as "pincode"',
      'Return JSON with key as "delivery_preferences"',
      'Return JSON with key as "time_slot"'
  ]
  json_keys = [
      'house_number',
      'gated_community',
      'area',
      'landmarks',
      'city',
      'state',
      'pincode',
      'delivery_preferences',
      'time_slot'
  ]

  for i, question in enumerate(questions):
    count = 0
    while True:
      count+=1
      print('Bot: ' + ('Sorry I couldnt get that. ' if count>1 else '' ) + question, end='\n')
      #voice response
      response = record_stt()
      
      #response = input("User: ")
      prompt = intro_prompt + f'''
      This is the question asked to the user: {question + json_format[i]}
      This is the response received from user : {response}
      ### STRICT INSTRUCTIONS ###
      You have to extract the information from the user's response according to the question.
      You have to give the JSON containing the information given by the user.
      You MUST give only the JSON as plain text. I DON'T WANY ANY OTHER WORD IN YOUR RESPONSE. Don't change the spelling of words used.
      If you can't retrieve the information or the user hasn't mentioned the information in the response, give the key as "Not Mentioned", remember this point this is very important, we don't want wrong information in our database, make sure to respond not mentioned if you don't get the information properly.
      '''
      response = llm(prompt)
      print(response,end='\n')
      
      json_data = json.loads(response)
      info_json[json_keys[i]] = 'Not Mentioned'
      if json_data[json_keys[i]] != 'Not Mentioned':
          info_json[json_keys[i]] = json_data[json_keys[i]]
          break



elif loc_type.strip() == 'Village':
  questions = [
      'Thank you. Now we can move on to collect your address. Can you please provide your House number or name of house or some other identity of the house',
      'Can you please provide the name of the society or road where it is situated?',
      'Can you tell me about any landmarks nearby, so that our delivery agents can find you more easily? Since you are in a village proper landmarks will be a great help.',
      'What is the name of the village?',
      'What is the state?',
      'What is the pincode?',
      'Can you provide any delivery prefrences for example "Leave the package at door or with guard" if any',
      'Can you provide the prefered time slot of your availability to collect the delivery'
  ]
  json_format = [
      'Return JSON with key as "house_identity"',
      'Return JSON with key as "society_or_road"',
      'Return JSON with key as "landmarks" and value should be array containing all the landmarks',
      'Return JSON with key as "village"',
      'Return JSON with key as "state"',
      'Return JSON with key as "pincode"',
      'Return JSON with key as "delivery_preferences"',
      'Return JSON with key as "time_slot"'
  ]
  json_keys = [
      'house_identity',
      'society_or_road',
      'landmarks',
      'village',
      'state',
      'pincode',
      'delivery_preferences',
      'time_slot'
  ]

  for i, question in enumerate(questions):
    count = 0
    while True:
      count+=1
      print('Bot: ' + ('Sorry I couldnt get that. Please remember that I am an AI bot, it will be helpful for me if you answer the question as short as possible' if count>1 else '' ) + question, end='\n')
      
      #voice response
      response = record_stt()
      
      #response = input("User: ")
      prompt = intro_prompt + f'''
      This is the question asked to the user: {question + json_format[i]}
      This is the response received from user : {response}
      ### STRICT INSTRUCTIONS ###
      You have to extract the information from the user's response according to the question.
      You have to give the JSON containing the information given by the user.
      You MUST give only the JSON as plain text. I DON'T WANY ANY OTHER WORD IN YOUR RESPONSE. Don't change the spelling of words used.
      If you can't retrieve the information or the user hasn't mentioned the information in the response, give the key as "Not Mentioned", remember this point this is very important, we don't want wrong information in our database, make sure to respond not mentioned if you don't get the information properly.
      '''
      response = llm(prompt)
      print(response,end='\n')
      
      json_data = json.loads(response)
      info_json[json_keys[i]] = 'Not Mentioned'
      if json_data[json_keys[i]] != 'Not Mentioned':
          info_json[json_keys[i]] = json_data[json_keys[i]]
          break


elif loc_type.strip() == 'Another type of location':
  questions = [
      'Thank you. Now we can move on to collect your address. Can you please provide your House number? or name of house or some other identity of the house',
      'Can you please provide the name of the society or road?',
      'What is the name of the area or locality where it is situated?',
      'Can you tell me about any landmarks nearby, so that our delivery agents can find you more easily?',
      'What is the name of the city or town?',
      'What is the state?',
      'What is the pincode?',
      'Can you provide any delivery prefrences for example "Leave the package at door or with guard" if any',
      'Can you provide the prefered time slot of your availability to collect the delivery'
  ]
  json_format = [
      'Return JSON with key as "house_number"',
      'Return JSON with key as "society_or_road"',
      'Return JSON with key as "area"',
      'Return JSON with key as "landmarks" and value should be array containing all the landmarks',
      'Return JSON with key as "city"',
      'Return JSON with key as "state"',
      'Return JSON with key as "pincode"',
      'Return JSON with key as "delivery_preferences"',
      'Return JSON with key as "time_slot"'
  ]
  json_keys = [
      'house_number',
      'society_or_road',
      'area',
      'landmarks',
      'city',
      'state',
      'pincode',
      'delivery_preferences',
      'time_slot'
  ]

  for i, question in enumerate(questions):
    count = 0
    while True:
      count+=1
      print('Bot: ' + ('Sorry I couldnt get that. ' if count>1 else '' ) + question, end='\n')
      
      #voice response
      response = record_stt()
      
      #response = input("User: ")
      prompt = intro_prompt + f'''
      This is the question asked to the user: {question + json_format[i]}
      This is the response received from user : {response}
      ### STRICT INSTRUCTIONS ###
      You have to extract the information from the user's response according to the question.
      You have to give the JSON containing the information given by the user.
      You MUST give only the JSON as plain text. I DON'T WANY ANY OTHER WORD IN YOUR RESPONSE. Don't change the spelling of words used.
      If you can't retrieve the information or the user hasn't mentioned the information in the response, give the key as "Not Mentioned", remember this point this is very important, we don't want wrong information in our database, make sure to respond not mentioned if you don't get the information properly.
      '''
      response = llm(prompt)
      print(response,end='\n')
      
      json_data = json.loads(response)
      info_json[json_keys[i]] = 'Not Mentioned'
      if json_data[json_keys[i]] != 'Not Mentioned':
          info_json[json_keys[i]] = json_data[json_keys[i]]
          break




Bot: Could you please tell me if you live in an apartment, a gated community, a village, or another type of location?
Recording...
Finished recording.
Bot: Thank you. Now we can move on to collect your address. Can you please provide your House number or name of house or some other identity of the house
Recording...
Finished recording.
{
  "house_identity": "House number thirteen by sixty-six and name of house is Vivek Khar"
}
Bot: Can you please provide the name of the society or road where it is situated?
Recording...
Finished recording.
```json
{"society_or_road":"GT Road"}
```


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [14]:
address_update_question = 'Bot: This is the information you provided, I will repeat them just for verification purposes, if you want to update anything then please let me know. The data you provided is as follows:' + str(info_json) + '. If you want to change anything then please let me know in which data do you want to make changes.'
print(address_update_question, end='\n')

# voice response
address_update_response = record_stt()

#address_update_response = input("User: ")
address_update_prompt = intro_prompt + f'''
This is the question asked to user: {address_update_question}
The response from user: {address_update_response}
This is the information in JSON format: {info_json}
###INSTRUCTIONS###
You have to look at user's response and determine if they want to update any information.
If they dont want to update anything RETURN "NO" as your response, REMEMBER ONLY "NO", I WANT NO OTHER WORD IN YOUR RESPONSE.
If they want to update anything the a text in format of PYTHON LIST containing all the fields that need to be updated. For eg. if user wants to update his area and landmarks then return ["area", "landmarks"], else if user wants to update his pincode only then return ["pincode"]. NO OTHER WORD OR SENTENCE SHOULD BE THERE IN YOUR RESPONSE OTHER THAN THIS PYTHON LIST.
'''
response = llm(address_update_prompt)
print(response)
if response.strip() != 'NO':
  try:
    update_list = ast.literal_eval(response)
    for i in update_list:
      question = f'I heard you want to update your {i}. Please provide the updated {i}.'
      print(question, end='\n')
      
      #voice response
      response = record_stt()
      
      #response = input("User: ")
      prompt = intro_prompt + f'''
      The user had previously given this information: {{'{i}':'{info_json[i]}'}}
      But now he wants to update: {i}
      The question asked: {question}
      The response from user: {response}
      ### STRICT INSTRUCTIONS ###
      You have to extract the upadted information from response, and return it in form of JSON exactly it was before.
      You MUST give only the JSON as plain text. I DON'T WANY ANY OTHER WORD IN YOUR RESPONSE. Don't change the spelling of words used
      '''
      response = llm(prompt)
      # print(response)
      json_data = json.loads(response)
      info_json[i] = json_data[i]

  except:
    pass
print('Bot: Thank you for providing the information.', end='\n')
print(info_json, end='\n')
info_json['contact'] = contact_json

Bot: This is the information you provided, I will repeat them just for verification purposes, if you want to update anything then please let me know. The data you provided is as follows:{'location_type': 'Village', 'house_identity': 'House number thirteen by sixty-six and name of house is Vivek Khar'}. If you want to change anything then please let me know in which data do you want to make changes.
Recording...
Finished recording.
NO
Bot: Thank you for providing the information.
{'location_type': 'Village', 'house_identity': 'House number thirteen by sixty-six and name of house is Vivek Khar'}


In [15]:
info_json

{'location_type': 'Village',
 'house_identity': 'House number thirteen by sixty-six and name of house is Vivek Khar',
 'contact': {'name': 'Vivek Kumar Gupta'}}